<a href="https://colab.research.google.com/github/SeonUI/ML_CS229/blob/main/4_Soft_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

##Support Vector Machines

Calculate **Hyperplane (Support Vector)** that divides two outputs

- Problem: Binary classification $x \mapsto \{-1,1\}$

- Goal: Increase Functional margin of hyperplane (Geometric margin)

- Hyperplane: $(w,b)$  ($w^Tx+b = 0$)

- $h_{w, b}(x) = g(w^Tx+b)$

- $g(z) = 1\space \text{or}\space-1$



**Functional margin**

- Distance between hyperplane and example $(x^{(i)},y^{(i)})$

$$
\hat{\gamma}^{(i)} = y^{(i)}(w^Tx^{(i)}+b) \\
\hat{\gamma} = \min \hat{\gamma}^{(i)}
$$



**Geometrical margin**

- Normalized functional margin

$$
{\gamma}^{(i)} = \frac{y^{(i)}(w^Tx^{(i)}+b)}{||w||} \\
\gamma = \min \gamma^{(i)}
$$

###How to study
Set closest functional margin is equal to zero $y^{(i)}(w^Tx^{(i)}+b) = 1$ ...
$$
{\gamma}^{(i)} = \frac{1}{||w||}
$$

- Minimize $||w||$
- subject to $y^{(i)}(w^Tx^{(i)}+b) \ge 1$


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

np.random.seed(0)
N_samples = 50
n_features = 2
n_classes = 2
X, Y = make_blobs(n_samples=N_samples, n_features=n_features, centers=n_classes,
                  cluster_std=1.5, random_state=42)

# iris = load_iris()
# X = iris.data
# Y = iris.target

Y = np.where(Y == 0, -1, 1)
Y = Y.reshape(-1, 1)

std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)

plt.figure(figsize=(6, 5))
plt.scatter(X[:, 0], X[:, 1], c=Y.flatten(), label='Data', alpha=0.8)
plt.title('Synthetic Data for Softmax Regression (3 Classes)')
plt.xlabel('X1')
plt.ylabel('X2')
plt.colorbar(label='Class Label')
plt.legend()
plt.grid(True)
plt.show()

# Split data into training and testing sets
# X[training_set, features], Y[training_set]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###Lagrange Multiplier
1. Transform to Lagrangian
    - Minimize $||w||$
    - subject to $y^{(i)}(w^Tx^{(i)}+b) \ge 1$

$$
𝓛(w,b,\alpha) = \frac12 ||w||^2 - \sum_{i=1}^m \alpha_i\left[y^{(i)}(w^Tx^{(i)}+b) -1 \right]
\\ \text{s.t. } \alpha_i \ge 0
$$

2. Derivations equal zero
$$
\nabla_w 𝓛(w,b,\alpha) = w-\sum_{i=1}^m \alpha_i y^{(i)}x^{(i)} = 0 \quad \\ \therefore w = \sum_{i=1}^m \alpha_i y^{(i)}x^{(i)}
\\
\frac{∂}{\partial b} = \sum_{i=1}^m \alpha_i y^{(i)} = 0
$$

3. Substitute primary equation with above
$$
\max\sum_i\alpha_i -\frac12\sum_i\sum_j\alpha_i\alpha_jy^{(i)}y^{(j)} \left< x^{(i)}, x^{(j)} \right> \quad \\ \text{s.t. } \alpha_i \ge 0, \quad
\sum_i \alpha_i y^{(i)}=0
$$

4. Learn for alpha
5. Get w with $w = \sum_{i=1}^m \alpha_i y^{(i)}x^{(i)}$

### $L_1$ norm soft margin SVM - Regularization
- cost xi, constant C

$$
\min _{\gamma, w, b}  \frac{1}{2}\|w\|^2+C \sum_{i=1}^m \xi_i \\
\text { s.t. } y^{(i)}\left(w^T x^{(i)}+b\right) \geq 1-\xi_i, \\ i=1, \ldots, m  \quad \xi_i \geq 0, \quad i=1, \ldots, m
$$

- same process.. turns to below
$$
W(\alpha) = \max\sum_i\alpha_i -\frac12\sum_i\sum_j\alpha_i\alpha_jy^{(i)}y^{(j)} \left< x^{(i)}, x^{(j)} \right> \quad \\ \text{s.t. } 0 \le \alpha_i \le C, \quad
\sum_i \alpha_i y^{(i)}=0
$$
→ Only difference is constraint of a_i **(why)**

###SMO Algorithm - How to learn $\alpha$

- Sequntial minimal optimization algorithm

1. Choose two alphas to optimize $\alpha_1, \alpha_2$
    - Want to optimize α one by one sequentially,
   but, because of this equation, $\sum_i \alpha_i y^{(i)}=0$ equation, optimize two by one.

2. Find maximum value of $W(\alpha) = W\left((\zeta-\alpha_2 y^{(2)}) y^{(1)}, \alpha_2, ... \alpha_m \right)$
    - it's just Quadratic function
    - $\alpha_1y^{(1)} + \alpha_2 y^{(2)} = \zeta$

3. Clipping: Alpha should be inside of (0, C)

4. Repeat until Convergence


### Choosing two alphas

1. Select alpha1 violates Karush-Kuhn-Tucker (KKT) conditions
    - $0 \ge \alpha_i \ge C$
    - $\alpha_i y_i = 1 (apply tol)

2. Select alpha2 maximizes next eqn.
    - |E_1 - E_2|
    - $E_1 = pred(X^{(1)}) - y^{(1)}$

### Updating two alphas (+ bias)
1. Calculate kernel difference $\eta$
    - $\eta = \left<x_1, x_1 \right> + \left<x_2, x_2 \right> -2\left<x_1, x_2 \right> $

2. Calculate new alphas
    - $\alpha_{2\text{new}} := \alpha_2 + y_2\frac{E_1 - E_2}{\eta}$

    - $\alpha_1 := \alpha_1 + y_1y_2d_2$

    - $d_2 = \alpha_2 - \alpha_{2\text{new}}$

3. Clipping alphas
    - clip alphas so that both alphas fit in [0,C]
    - consider alpha2's value when clipping alpha1

4. Calculate new bias
    - $b_\text{new} := b - E_1 - y_1d_1\left<x_1, x_1 \right> - y_2d_2\left<x_1, x_2 \right>$

    - $b_\text{new} := b - E_2 - y_1d_1\left<x_1, x_2 \right> - y_2d_2\left<x_2, x_2 \right>$

    - Choose b in the range [0,C].
     If both all in the range, take average

### Kernel
Can use feature space $ \phi(x) = \{x_1, x_2, x_1x_2, x_1^2 ... \}$

We replace all $ \left<x, z \right> = x^Tz $ with $K(x,z)$
$$
\left<x, z \right> = x^Tz \text{(linear Kernel)} \\
K(x,z) = \phi(x)^T\phi(z)
$$

In [ ]:
def linear_kernel(x,z):
    return np.dot(x, z.T)

# return Prediction using dual form -> np.array[# samples]
def predict(X_val, Alpha, Y, X_train, b, kernel) -> np.array:
    # X_val can be a single sample (n_features,) or a batch of samples (n_samples, n_features)
    if X_val.ndim == 1:
        X_val_2d = X_val.reshape(1, -1)
    else:
        X_val_2d = X_val

    m = X_train.shape[0]
    preds = np.zeros((X_val_2d.shape[0], 1))

    for i in range(X_val_2d.shape[0]):
        prediction = 0
        for j in range(m):
            prediction += Alpha[j] * Y[j] * kernel(X_val_2d[i], X_train[j])
        preds[i] = prediction + b
    return preds

# return two alphas
def choose_alpha(Alpha, X_train, Y_train, b, C, tol, kernel) -> tuple[int,int]:
    m = X_train.shape[0]

    for i in range(m):
        pred_xi = predict(X_train[i], Alpha, Y_train, X_train, b, kernel)
        y_i = Y_train[i]
        alpha_i = Alpha[i]

        # KKT conditions for alpha_i
        kkt_violation = False
        if alpha_i <= 0:
            kkt_violation = True
        elif alpha_i >= C:
            kkt_violation = True
        elif abs(y_i * pred_xi - 1) > tol:
            kkt_violation = True

        if kkt_violation:
            alpha1_idx = i
            alpha2_idx = -1
            error_i = pred_xi - y_i
            max_error = tol
            # Find second alpha - find max error diff j
            for j in range(m):
                if i == j: continue
                pred_xj = predict(X_train[j], Alpha, Y_train, X_train, b, kernel)
                error_j = pred_xj - Y_train[j]
                error_ij = abs(error_i - error_j)

                if error_ij > max_error:
                    alpha2_idx = j
                    max_error = error_ij

            if alpha2_idx != -1:
                return alpha1_idx, alpha2_idx

    return -1, -1

def update_alpha(X_train, Y_train, Alpha, b, alpha1_idx, alpha2_idx, C, kernel, tol_alpha=1e-5):
    alpha1 = Alpha[alpha1_idx].copy()
    alpha2 = Alpha[alpha2_idx].copy()
    y1, y2 = Y_train[alpha1_idx], Y_train[alpha2_idx]
    x1, x2 = X_train[alpha1_idx], X_train[alpha2_idx]

    # Calculate E_i and E_j using the current model's prediction
    E1 = predict(x1, Alpha, Y_train, X_train, b, kernel) - y1
    E2 = predict(x2, Alpha, Y_train, X_train, b, kernel) - y2

    # Calculate eta, kernel distance between i, j
    K11 = kernel(x1, x1)
    K22 = kernel(x2, x2)
    K12 = kernel(x1, x2)
    eta = K11 + K22 - 2 * K12
    if eta <= 0:
        return b

    # Calculate L, H
    #alpha1 - alpha2 = const
    if y1 != y2:
        L = max(0, alpha2 - alpha1)
        H = min(C, C + alpha2 - alpha1)
    #alpha1 + alpha2 = const
    elif y1 == y2:
        L = max(0, alpha1 + alpha2 - C)
        H = min(C, alpha1 + alpha2)

    # Calculate new alpha1, 2
    alpha2_new_unclipped = alpha2 + y2 * (E1 - E2) / eta
    alpha2_new = np.clip(alpha2_new_unclipped, L, H)
    if abs(alpha2_new - alpha2) < tol_alpha:
        return b

    alpha1_new = alpha1 + y1 * y2 * (alpha2 - alpha2_new)

    # Update
    Alpha[alpha1_idx] = alpha1_new
    Alpha[alpha2_idx] = alpha2_new

    b1 = b - E1 - y1 * K11 * (alpha1_new - alpha1) - y2 * K12 * (alpha2_new - alpha2)
    b2 = b - E2 - y1 * K12 * (alpha1_new - alpha1) - y2 * K22 * (alpha2_new - alpha2)

    if 0 < alpha1_new < C:
        b_new = b1
    elif 0 < alpha2_new < C:
        b_new = b2
    else: # Both alpha_i_new and alpha_j_new are at their bounds (0 or C), take average
        b_new = (b1 + b2) / 2.0
    return b_new

def get_w(Alpha, X_train, Y_train) -> np.array:
    return np.sum(Alpha * Y_train * X_train, axis=0).reshape(-1, 1)

def svm(X_train, Y_train, limit, C=1.0, tol=1e-3, kernel=linear_kernel):
    m, n = X_train.shape
    Alpha = np.zeros((m, 1))
    b = 0

    for i in range(limit):
        alpha1_idx, alpha2_idx = choose_alpha(Alpha, X_train, Y_train, b, C, tol, kernel)

        if (alpha1_idx == -1) and (alpha2_idx == -1):
            break

        b = update_alpha(X_train, Y_train, Alpha, b, alpha1_idx, alpha2_idx, C, kernel)

    w = get_w(Alpha, X_train, Y_train)
    return w, b




w, b = svm(X_train, Y_train, 1000)
print (w, b)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume w, b, X_train, Y_train are available from the kernel state

# Select the same two features used in the initial scatter plot (X[:, 0], X[:, 3])
feature_idx1 = 0
feature_idx2 = 1

# Extract the scalar values for w components and b
w0_val = w[feature_idx1].item()
w3_val = w[feature_idx2].item()
b_val = b.item()

# Plot the training data
plt.figure(figsize=(8, 6))
plt.scatter(X_train[:, feature_idx1], X_train[:, feature_idx2], c=Y_train.flatten(), cmap='bwr', label='Training Data', alpha=0.8)

# Define the range for the decision boundary line
x_min, x_max = X_train[:, feature_idx1].min() - 0.5, X_train[:, feature_idx1].max() + 0.5
xx = np.linspace(x_min, x_max, 100)

# Calculate the decision boundary for the selected features
# The equation for the decision boundary is w[feature_idx1]*x_feature1 + w[feature_idx2]*x_feature2 + b = 0
# Solving for x_feature2: x_feature2 = (-w[feature_idx1]*x_feature1 - b) / w[feature_idx2]

epsilon = 1e-6 # Small value to prevent division by zero
if abs(w3_val) < epsilon: # If w3 is negligible, the decision boundary is primarily vertical in this 2D projection
    if abs(w0_val) < epsilon: # Both w0 and w3 are negligible for the projection
        plt.text((x_min + x_max)/2, X_train[:, feature_idx2].mean(), "Cannot plot clear boundary (w0 and w3 near zero)", horizontalalignment='center')
    else:
        # Plot a vertical line. Y-range is defined by the data's Y-axis range.
        plt.axvline(x=(-b_val / w0_val), color='k', linestyle='--', label='Decision Boundary')
        plt.ylim(X_train[:, feature_idx2].min() - 0.5, X_train[:, feature_idx2].max() + 0.5)
else:
    # Calculate the y-values for the decision boundary line
    yy = (-w0_val * xx - b_val) / w3_val
    plt.plot(xx, yy, 'k--', label='Decision Boundary')

plt.title(f'SVM Decision Boundary (2D Projection of Features {feature_idx1+1} and {feature_idx2+1})')
plt.xlabel(f'Feature {feature_idx1+1}')
plt.ylabel(f'Feature {feature_idx2+1}')
plt.colorbar(label='Class Label')
plt.legend()
plt.grid(True)
plt.show()